# Capstone: Financial Research Copilot

## Step 0: Setup and Configuration
Importing necessary libraries (yfinance, pandas, google-generativeai) and configuring the Google Gemini client using the API key from `.env`.

In [1]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import google.generativeai as genai

c:\Users\yuhes\Courses\Google & Kaggle Gen AI\Capstone\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Load environment variables
load_dotenv()

# 2. Get API Key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("Error: GOOGLE_API_KEY not found in .env file.")
else:
    print("API Key loaded successfully.")
    
    # 3. Configure the client
    try:
        genai.configure(api_key=api_key)
        print("Google GenAI Client configured successfully.")
    except Exception as e:
        print(f"Error configuring client: {e}")

API Key loaded successfully.
Google GenAI Client configured successfully.


## Step 1.1: Price History Tool
Defining `get_asset_price_history` to fetch OHLCV data for Stocks and ETFs using `yfinance`. This tool returns structured JSON data that the agent can analyze.


In [3]:
from typing import Dict, Any

def get_asset_price_history(symbol: str, asset_type: str = "equity", period: str = "1mo") -> Dict[str, Any]:
    """
    Fetch historical OHLCV price data for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL', 'BTC-USD').
        asset_type (str): Type of asset ('equity', 'crypto', 'etf'). Defaults to 'equity'.
        period (str): Data period to download (e.g., '1mo', '3mo', '1y'). Defaults to '1mo'.
        
    Returns:
        dict: Structured data containing status and price history.
    """
    try:
        # Handle crypto symbols for yfinance (usually needs -USD suffix if not provided)
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
            symbol = f"{symbol}-USD"
            
        ticker = yf.Ticker(symbol)
        
        # Fetch history
        hist = ticker.history(period=period)
        
        if hist.empty:
            return {
                "status": "error",
                "message": f"No data found for {symbol}. Check ticker or period."
            }
            
        # Format data for the agent (convert to list of dicts)
        # We limit to the last 60 records to keep context small if period is long
        data_records = []
        for date, row in hist.tail(60).iterrows():
            data_records.append({
                "date": date.strftime('%Y-%m-%d'),
                "close": round(row['Close'], 2),
                "volume": int(row['Volume'])
            })
            
        return {
            "status": "success",
            "symbol": symbol,
            "currency": ticker.info.get('currency', 'USD'),
            "data": data_records
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [4]:
# --- Testing the tool  ---
print("Testing Tool 1.1 with AAPL...")
result = get_asset_price_history("AAPL", period="5d")
print(result)


Testing Tool 1.1 with AAPL...
{'status': 'success', 'symbol': 'AAPL', 'currency': 'USD', 'data': [{'date': '2025-11-20', 'close': np.float64(266.25), 'volume': 45823600}, {'date': '2025-11-21', 'close': np.float64(271.49), 'volume': 59030800}, {'date': '2025-11-24', 'close': np.float64(275.92), 'volume': 65585800}, {'date': '2025-11-25', 'close': np.float64(276.97), 'volume': 46914200}, {'date': '2025-11-26', 'close': np.float64(277.55), 'volume': 33431400}]}


## Step 1.2: Fundamentals Tool
Defining `get_asset_fundamentals` to retrieve key financial metrics (P/E, Market Cap, Sector) using `yfinance`. This helps the agent assess value and risk.


In [5]:
def get_asset_fundamentals(symbol: str, asset_type: str = "equity") -> Dict[str, Any]:
    """
    Fetch fundamental data (P/E, Market Cap, Sector, Summary) for an asset.
    
    Args:
        symbol (str): Ticker symbol (e.g., 'AAPL').
        asset_type (str): 'equity' or 'etf'. (Crypto has limited fundamentals in yf).
        
    Returns:
        dict: Key fundamentals or error message.
    """
    try:
        if asset_type.lower() == 'crypto' and not symbol.endswith('-USD'):
             symbol = f"{symbol}-USD"

        ticker = yf.Ticker(symbol)
        info = ticker.info
        
        # Extract only the most relevant fields to save context tokens
        fundamentals = {
            "symbol": symbol,
            "name": info.get("shortName", "Unknown"),
            "sector": info.get("sector", "Unknown"),
            "industry": info.get("industry", "Unknown"),
            "market_cap": info.get("marketCap", "N/A"),
            "trailing_pe": info.get("trailingPE", "N/A"),
            "forward_pe": info.get("forwardPE", "N/A"),
            "dividend_yield": info.get("dividendYield", "N/A"),
            "fifty_two_week_high": info.get("fiftyTwoWeekHigh", "N/A"),
            "fifty_two_week_low": info.get("fiftyTwoWeekLow", "N/A"),
            "summary": info.get("longBusinessSummary", "No summary available")[:300] + "..." # Truncate summary
        }
        
        return {
            "status": "success",
            "data": fundamentals
        }
        
    except Exception as e:
        return {"status": "error", "message": str(e)}


In [6]:
# --- Test the tool immediately ---
print("Testing Tool 1.2 with MSFT...")
fund_result = get_asset_fundamentals("MSFT")
print(fund_result)

Testing Tool 1.2 with MSFT...
{'status': 'success', 'data': {'symbol': 'MSFT', 'name': 'Microsoft Corporation', 'sector': 'Technology', 'industry': 'Software - Infrastructure', 'market_cap': 3608802230272, 'trailing_pe': 34.481533, 'forward_pe': 32.47492, 'dividend_yield': 0.75, 'fifty_two_week_high': 555.45, 'fifty_two_week_low': 344.79, 'summary': "Microsoft Corporation develops and supports software, services, devices, and solutions worldwide. The company's Productivity and Business Processes segment offers Microsoft 365 Commercial, Enterprise Mobility + Security, Windows Commercial, Power BI, Exchange, SharePoint, Microsoft Teams, Security a..."}}


## Step 1.3: Technical Analysis Tool
Defining `calculate_technical_indicators` to compute Volatility (Standard Deviation) and Moving Averages. This gives the agent quantitative data to assess risk.


In [7]:
def calculate_technical_indicators(symbol: str, period: str = "3mo") -> Dict[str, Any]:
    """
    Calculate technical indicators (Volatility, SMA) for a symbol.
    
    Args:
        symbol (str): Ticker symbol.
        period (str): Data lookback period (default '3mo').
        
    Returns:
        dict: Calculated metrics including annualized volatility and current trend.
    """
    try:
        # We reuse our first tool logic to get the raw dataframe usually, 
        # but calling yfinance directly here is cleaner for the standalone tool.
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period=period)
        
        if hist.empty:
             return {"status": "error", "message": "No data for calculations."}
        
        # 1. Calculate Daily Returns
        hist['Returns'] = hist['Close'].pct_change()
        
        # 2. Annualized Volatility (Standard Deviation of returns * sqrt(252 trading days))
        volatility = hist['Returns'].std() * np.sqrt(252)
        
        # 3. Simple Moving Average (50-day) - requiring enough data
        if len(hist) >= 50:
            sma_50 = hist['Close'].rolling(window=50).mean().iloc[-1]
        else:
            sma_50 = "Not enough data"
            
        current_price = hist['Close'].iloc[-1]
        
        # Determine trend
        trend = "Neutral"
        if isinstance(sma_50, (int, float)):
            if current_price > sma_50:
                trend = "Uptrend (Above 50d SMA)"
            else:
                trend = "Downtrend (Below 50d SMA)"
        
        return {
            "status": "success",
            "symbol": symbol,
            "current_price": round(current_price, 2),
            "annualized_volatility": round(volatility * 100, 2), # as percentage
            "sma_50": round(sma_50, 2) if isinstance(sma_50, float) else sma_50,
            "trend": trend
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}


In [8]:
# --- Test the tool ---
print("Testing Tool 1.3 with NVDA...")
tech_result = calculate_technical_indicators("NVDA")
print(tech_result)

Testing Tool 1.3 with NVDA...
{'status': 'success', 'symbol': 'NVDA', 'current_price': np.float64(180.26), 'annualized_volatility': np.float64(37.57), 'sma_50': np.float64(186.81), 'trend': 'Downtrend (Below 50d SMA)'}
